In [31]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import Callback


%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [29]:
YEAR = range(2012,2021)
data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(YEAR) + '.csv.gz',
                   compression= 'gzip', low_memory= False)



InvalidURL: URL can't contain control characters. '/nflverse/nflverse-data/releases/download/pbp/play_by_play_range(2012, 2021).csv.gz' (found at least ' ')

Data Cleaning

In [50]:
##get data
##data is already from 2012-2020
data = pd.read_csv('cleaneddata.csv') # pandas data frame


##preprocessing

##filter to only patriots
data = data.loc[(data['home_team'] == 'NE') | (data['away_team'] == 'NE')]

##removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']
data = data[data['posteam'] == 'NE']
data = data.dropna(subset=['down'])
data = data.dropna(subset=['play_type'])
data = data[(data['play_type'] == 'pass') | (data['play_type'] == 'run')]
# Look into creating specific year and months, and training and testing on specific years/gmaes. 


# Extract year and month using string slicing
data['year'] = data['game_id'].str[:4].astype(int)
data['month'] = data['game_id'].str[5:7].astype(int)

# Create a new 'date' column with the 1st day of each extracted year and month


# Come back to this when we want to specify split spots  
# def parse_date(x):
#     parts = x.split('_')
#     year = int(parts[0])
#     month = int(parts[1])
#     print(month)
#     return pd.Timestamp(year=int(year), month=month), day=1)

# Apply the function to the 'game_date' column
# data['date'] = data['game_id'].apply(parse_date)
# # Now you can sort by this 'date' column if needed
# data.sort_values(by='date', inplace=True)

# # And use the 'date' column for splitting your data, work with in the future to pick specific dates 
# #Right now not working because the months in the game_id are 01-12, when for pd.TimeStamp they need to be in 1-12. 
# split_date = pd.Timestamp(year=2018, month=12, day=1)  # Replace YYYY and MM with your split year and month
# train_data = data[data['date'] < split_date]
# test_data = data[data['date'] >= split_date]

 
##fixes playtype to be strictly pass or run 
##lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'

# columns are the strings 
# things to encode from strings to 1s and zeros
# season_type, 
columnsKeep = ['game_id', 'year','month','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success', 'wp']

data = data[columnsKeep]

# print(data.head(5))

data.to_csv('finalized_data.csv', index=False)

/tmp/ipykernel_66276/2671775745.py:3: DtypeWarning: Columns (45,179,180,189,190,197,198,203,204,205,206,218,219,220,222,224,226,253,254,255,260,262,263,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cleaneddata.csv') # pandas data frame


In [55]:
print(len(data))

# Count occurrences of each play type
runCount = (data['play_type'] == 'run').sum()
passCount = (data['play_type'] == 'pass').sum()

print("Number of 'run' plays:", runCount)
print("Number of 'pass' plays:", passCount)

9565
Number of 'run' plays: 3941
Number of 'pass' plays: 5624


In [56]:
##check for nan values in data
for index, row in data.iterrows():
    if pd.isnull(row['play_type']):  # Check if the value in the 'down' column is NaN
        print(row['desc'])  # Print the value in the 'desc' column if 'down' is NaN

# wind and stadium had a lot of null 
## check for missing values in data
selected_features = ['down', 'drive', 'ydstogo', 'yardline_100', 'half_seconds_remaining', 'game_seconds_remaining','wp']
missing_values_features = data[selected_features].isnull().sum()
print("Missing values in selected features:")
print(missing_values_features)

# Check for missing values in the target
missing_values_target = data['play_type'].isnull().sum()
print("\nMissing values in target (play_type):", missing_values_target)


Missing values in selected features:
down                      0
drive                     0
ydstogo                   0
yardline_100              0
half_seconds_remaining    0
game_seconds_remaining    0
wp                        0
dtype: int64

Missing values in target (play_type): 0


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = data[['down','drive', 'ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'wp']]
target = data['play_type'].map({'pass': 0, 'run': 1})

# Initialize the scaler 
scaler = StandardScaler() 

# Fit the scaler on the features and transform them 
features_scaled = scaler.fit_transform(features)

# Reshaping input data for LSTM [samples, time steps, features]
# Here, we assume each sample is a single time step
features_scaled = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))

# Talk to Nicolai and Ulrich if they think this is a good idea to split it up into drives as our time stamp. 
# 1 indicates that each sample consists of 1 timestep. This is a key detail: 
# it means you are treating each play as a separate, single-timestep sequence. T
# This is a simplification, as ideally, plays within a single drive might be treated as sequences to capture their temporal dependencies.

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, shuffle = False)

#Initalize model
model = Sequential()

# input_shape corresponds to (time steps, features)
model.add(LSTM(units=50, return_sequences=True, input_shape=(1, features_scaled.shape[2])))
model.add(Dropout(0.2))  # Adding dropout to avoid overfitting

# Adding a second LSTM layer
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Adding the output layer
# Since binary classification (pass/run), we use 1 unit with a sigmoid activation function
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_split=0.1)

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}')



Epoch 1/30
108/108 [==============================] - 5s 12ms/step - loss: 0.6565 - accuracy: 0.6414 - val_loss: 0.6301 - val_accuracy: 0.6279
Epoch 2/30
108/108 [==============================] - 0s 4ms/step - loss: 0.6099 - accuracy: 0.6611 - val_loss: 0.6061 - val_accuracy: 0.6593
Epoch 3/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5986 - accuracy: 0.6718 - val_loss: 0.5926 - val_accuracy: 0.6697
Epoch 4/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5932 - accuracy: 0.6747 - val_loss: 0.5906 - val_accuracy: 0.6841
Epoch 5/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5894 - accuracy: 0.6738 - val_loss: 0.5859 - val_accuracy: 0.6893
Epoch 6/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5888 - accuracy: 0.6757 - val_loss: 0.5851 - val_accuracy: 0.6893
Epoch 7/30
108/108 [==============================] - 0s 4ms/step - loss: 0.5866 - accuracy: 0.6798 - val_loss: 0.5808 - val_accuracy: 0.6984
Epoch

In [58]:
# length of X train, and nulls in the x,train, x_test, y_train, y_test 
print(len(X_train))
nan_indices_X = np.isnan(X_train)
nan_count_X = np.sum(nan_indices_X)
print(nan_count_X)

nan_indices_X_test = np.isnan(X_test)
nan_count_X_test = np.sum(nan_indices_X_test)
print(nan_count_X_test)

nan_indices_y = np.isnan(y_train)
nan_count_y = np.sum(nan_indices_y)
print(nan_count_y)

nan_indices_Y_test = np.isnan(y_test)
nan_count_Y_test = np.sum(nan_indices_Y_test)
print(nan_count_Y_test)

print("Shape of X_train:", X_train.shape)

7652
0
0
0
0
Shape of X_train: (7652, 1, 7)


In [12]:
features = data[['down','drive', 'ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind']]
target = data['play_type'].map({'pass': 0, 'run': 1})

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['down','ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind'])
    ]
)

X_processed = preprocessor.fit_transform(features)

X_lstm = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y = target.values

X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['accuracy']
)



##history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

##test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
##print(f'Test Accuracy: {test_acc}')



# Assuming you have already defined X_test and y_test
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int).reshape(-1)

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc}')


for i in range(10):
    actual = 'pass' if y_test[i] == 0 else 'run'
    predicted = 'pass' if predictions[i] == 0 else 'run'
    print(f'Actual: {actual}, Predicted: {predicted}')

66/66 [==============================] - 0s 2ms/step
Epoch 1/50
237/237 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.5356 - val_loss: nan - val_accuracy: 0.5433
Epoch 2/50
237/237 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.5433
Epoch 3/50
237/237 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.5433
Epoch 4/50
237/237 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.5433
Epoch 5/50
237/237 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.5433
Epoch 6/50
237/237 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.5433
Epoch 7/50
237/237 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.5364 - val_loss: nan - val_accuracy: 0.